<h1> Entrenamiento y optimización de Modelos de Machine Learning </h1>
<h3> Proyecto final: </h3> 
<h4> Sergio Andres Rios Gomez </h4>

<h2> 1. Definición de objetivo </h2>
<p> Hacer uso eficiente de la información transaccional con el fin de optimizar los procesos internos al interior de la entidad  </p> 

<h2> 2. Contexto comercial</h2>
<p> Los sistemas de pago de bajo valor SPBV, son mecanismos de funcionamiento de la economía moderna, donde participa un originador (Pagador) y un beneficiario (receptor) de los recursos. Ante el alto volumen de pagos y la necesidad de automatizar estas transacciones nacen compañias especializadas en realizar toda esta intermediación. </p>

<p> La empresa SPBV se encarga de prestar los servicios tecnologicos y transaccionales a las Cooperativas de la red. </p>

<p> La data que será procesada en la presente entrega, corresponse a las transacciones de mensuales de la entidad administradora de pagos de bajo valor. La información contiene detalle de los grupos, canales, tipos de transacciones, montos transados y todo lo detallado con cada transacción</p>
 

<h2> 3. Problema comercial </h2>
<p> En la empress SPVB se generan miles de registros transacciones a diario, se pretende dar respuesta con la data del modelo a diferentes necesidades las cuales son: </p>

<li> 1. Crear un modelo para identificar y predecir las transaccio exitosas y con error</li>
<li> 2. Modelo para inferir el tipo de transacción que pueden realizar los usuarios de la red.</li>
<li> 3. Modelo de regresión lineal para predecir los retiros en efectivo y de esta manera provicionar el cash necesario para cubrir las necesidades de los usuarios.</li>

<h2> 4. Data Acquisition </h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
%matplotlib inline

In [ ]:
df = pd.read_csv('C:/Users/chech/PC Febrero 2023/1. Data Scients/CoderHouse/Data Scients/FinalProject/data/12. Dic22.csv')
df = pd.DataFrame(df)

In [ ]:
print("El tamaño del dataset es de: ", df.shape)

In [ ]:
df.dtypes

<h2> 5. Data Wrangling </h2> 

In [ ]:
# Missing Data 
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/ df.isnull().count().sort_values(ascending = False)*100)
missing_data = pd.concat([total, percent], axis=1, keys=['total','percent'])
missing_data.head(14)

<p> Se reduce en Data Frame, eliminando las columnas con más del 78% de datos faltantes, algunas que son redundantes pues brindan la misma información en los modelos y alfunos features que a consideración no generan valor.

In [ ]:
df_1 = df[['ID. Transaccion','Nombre Transaccion','BIN','ID. Canal','Nombre Canal','Error','Nombre Error',
           'Responsable Error','ID. Origen','Nombre Origen','ID. Grupo','Nombre Grupo','Valor','Fecha Transaccion','Codigo Entidad Orgien',
           'Cooperativa Terminal','ID. Terminal','Nombre Terminal','Serial Terminal','Tipo Documento',
           'Documento','Departamento','Ciudad']]

<p> Convertir el cambo fecha en dd-mm-aaaa

In [ ]:
# Convertir la columna 'Fecha Transaccion' al formato datetime
df_1['Fecha Transaccion'] = pd.to_datetime(df_1['Fecha Transaccion'])

# Extraer el día, mes y año y crear una nueva columna 'Fecha Formateada'
df_1['Fecha Transaccion'] = df_1['Fecha Transaccion'].dt.strftime('%d-%m-%Y')

In [ ]:
# Eliminar las filas con valores no finitos en la columna "Documento"
df_1 = df_1.dropna(subset=['Documento'])

# Convertir los valores en la columna "Documento" de tipo float64 a int64
df_1["Documento"] = df_1["Documento"].astype('int64')

<p> Las transacciones de consulta, registrar valor 0 (cero) en la columna "Valor", con el fin de no mostrar movimientos financieros en transacciones de consultas.

In [ ]:
df_1.loc[df_1['Nombre Transaccion'].str.contains('Consul'), 'Valor'] = 0

<p> En la columna valor, poner "0" (cero) a las transacciones que no fueron exitosas, esto con el fin de garantizar que no se sumen en la red valores que realmento no transaron.

In [ ]:
# Actualizar los valores de la columna "Valor" según la condición
df_1.loc[df_1['Nombre Error'] != 'Exito', 'Valor'] = 0

In [ ]:
# Eliminar filas con valores nulos en 'Departamento' y 'Ciudad'
df_1.dropna(subset=['Departamento', 'Ciudad'], inplace=True)

In [ ]:
# Convertir la columna "Departamento" a tipo de dato entero
df_1['Departamento'] = df_1['Departamento'].astype(int)

# Convertir la columna "Ciudad" a tipo de dato entero
df_1['Ciudad'] = df_1['Ciudad'].astype(int)

<h2> 6. Feature Engineering </h2>

<p> 6.1 Función para asignar valores en función de la columna "Error" y crear una nueva columna binaria que indique "1" si la transaccion fue exitosa o "0" si fue error

In [ ]:
def asignar_exito(row):
    if row["Error"] == 0 or row["Error"] == 900:
        return 1
    else:
        return 0
df_1["Exito1-0"] = df_1.apply(asignar_exito, axis=1)

<p> 6.2 Con el fin de categorizar las transacciones por rango del valor transado, se crea una nueva columna que indica en que rango esta cada transacción.

In [ ]:
def rangos(valor):
    if valor > 100000000:
        return '+ 100 m'
    elif valor > 50000000 and valor <= 100000000:
        return '50 m-100 m'
    elif valor > 20000000 and valor <= 50000000:
        return '20 m-50 m'
    elif valor > 5000000 and valor <= 20000000:
        return '5 m-20 m'
    elif valor > 1000000 and valor <= 5000000:
        return '1 m-5 m'
    elif valor > 200000 and valor <= 1000000:
        return '200-1 m'
    elif valor > 50000 and valor <= 200000:
        return '50-200'
    elif valor > 10000 and valor <= 50000:
        return '10-50'
    elif valor > 1 and valor <= 10000:
        return '1-10'
    else:
        return 'Cero'
df_1['NombreRango'] = df_1['Valor'].apply(rangos)

<h2> 7. Exploratory Data Analysis - EDA </h2>

<p> 7.1 Visualización de valores null en el data frame.

In [ ]:
(
    df_1.isnull()
    .melt().pipe(
        lambda i:(
            sns.displot(
                data = i,
                y = 'variable',
                hue = 'value',
                multiple = 'fill',
                aspect = 2
            )
        )
    )
)

<p> 7.2 Porcentaje de trasacciones exitosas Vs Error.

In [ ]:
suma_valores = df_1["Exito1-0"].value_counts()

# Crear la gráfica de torta
plt.figure(figsize=(6, 6))
plt.pie(suma_valores, labels=["Exito", "Error"], autopct='%1.1f%%', colors=['silver', 'red'], startangle=140)

plt.title("Estado de las transacciones ")

plt.show()

<p> 7.3 Cantidad de transacciones por rango. Lo cual nos da una idea muy acertada del comportamiento de las transacciones en terminos del valor $

In [ ]:
conteo_rangos = df_1['NombreRango'].value_counts()

# Crear la gráfica de barras
plt.figure(figsize=(10, 6))
plt.bar(conteo_rangos.index, conteo_rangos.values, color='orange')

# Personalizar la gráfica
plt.title('Distribución de Transacciones por Rango')
plt.xlabel('Rango')
plt.ylabel('Conteo')

# Rotar las etiquetas del eje x si es necesario
plt.xticks(rotation=45)
plt.show()

<p> 7.4 Cantidad de transacciones por canal.

In [ ]:
# Seleccionar la columna 'Nombre Transaccion'
nombre_transaccion = df_1['Nombre Canal']

# Contar las ocurrencias de cada valor
conteo_transacciones = nombre_transaccion.value_counts()

# Crear la gráfica de barras horizontales
plt.figure(figsize=(10, 6))
sns.barplot(x=conteo_transacciones.values, y=conteo_transacciones.index, color='blue')

# Personalizar la gráfica
plt.title('Distribución de la columna "Nombre Transaccion"')
plt.xlabel('Conteo')
plt.ylabel('Nombre Transaccion')

# Mostrar la gráfica
plt.show()

<p> 7.5 Cantidad de transacciones por cada tipo de transacción.

In [ ]:
nombre_transaccion = df_1['Nombre Transaccion']

# Contar las ocurrencias de cada valor
conteo_transacciones = nombre_transaccion.value_counts()

# Crear la gráfica de barras horizontales
plt.figure(figsize=(10, 6))
sns.barplot(x=conteo_transacciones.values, y=conteo_transacciones.index, color='green')

# Personalizar la gráfica
plt.title('Distribución de la columna "Nombre Transaccion"')
plt.xlabel('Conteo')
plt.ylabel('Nombre Transaccion')

# Mostrar la gráfica
plt.show()

<h2> Feature engineering - Segunda parte </h2>
<p> Antes de iniciar con el proceso de modelado, procedemos a continuar haciendo feature engineering, con el fin de tener el dataset lo mejor posible para obtener un buen modelo. 

In [ ]:
df_2 = df_1[['ID. Origen','ID. Canal','ID. Transaccion','BIN','Valor','Codigo Entidad Orgien',
             'Departamento', 'Ciudad', 'Error','Exito1-0', 'NombreRango','Tipo Documento','Documento']]
df_2

<h4> Label Encoding </h4>
<p> Transformar variables categóricas en variables numéricas. Consiste en asignar un número único a cada categoría en la columna categórica, convirtiendo así las etiquetas en valores enteros.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Columnas para aplicar Label Encoding
columns_to_encode = ['ID. Grupo', 'ID. Transaccion', 'BIN', 'NombreRango', 'Tipo Documento']

# Crear un objeto LabelEncoder
label_encoder = LabelEncoder()

# Aplicar Label Encoding a cada columna
for col in columns_to_encode:
    if col in df_2.columns:
        df_2[col] = label_encoder.fit_transform(df_2[col].astype(str))

In [ ]:
df_2

In [ ]:
# Seleccionar solo las columnas numéricas
columns_numeric = ['ID. Origen', 'ID. Canal', 'ID. Transaccion', 'BIN',
                   'Valor', 'Codigo Entidad Orgien', 'Departamento', 'Ciudad', 'Error',
                   'Exito1-0', 'NombreRango', 'Tipo Documento', 'Documento']

# Crear un DataFrame con las columnas numéricas
df_numeric = df_2[columns_numeric]

# Calcular la matriz de correlaciones
correlation_matrix = df_numeric.corr()

# Crear la visualización de la matriz de correlaciones
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matriz de Correlaciones')
plt.show()

<h2> 8. Seleccion de algoritmos apropiados </h2>

<p> 8.1 Crear un modelo para identificar y predecir las transaccio exitosas y con error </p>
<h3> Regresión logistica y Random forest </h3>

<p> 8.2 Modelo para inferir el tipo de transacción que pueden realizar los usuarios de la red.</p>
<h3> Random Forest</h3>

<p> 8.3 Modelo para predecir los retiros en efectivo y de esta manera provicionar el cash necesario para cubrir las necesidades de los usuarios.</p>

<h2> 9. Desarrollo del algoritmo </h2>

<p> 8.1 Crear un modelo para identificar y predecir las transaccio exitosas y con error </p>
<h2> Regresión Logística: </h2>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Hacer una copia del DataFrame df_2 y llamarla df_3
df_3 = df_2.copy()

# Sacar "Exito1-0" como target y las características
X = df_3.drop('Exito1-0', axis=1)
y = df_3['Exito1-0']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los features usando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo de Regresión Logística
# Aumentamos el número máximo de iteraciones
logistic_model = LogisticRegression(max_iter=1000)

# Definir los parámetros a probar para la optimización (en este caso solo C, el hiperparámetro de regularización)
param_grid = {'C': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Usar validación cruzada (5-fold) para encontrar el mejor parámetro
grid_search = GridSearchCV(logistic_model, param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

# Obtener el mejor parámetro y su desempeño
best_param = grid_search.best_params_
print('Mejor parámetro para C:', best_param)

# Entrenar el modelo con el mejor parámetro encontrado
best_logistic_model = LogisticRegression(C=best_param['C'], max_iter=1000)
best_logistic_model.fit(X_train_scaled, y_train)

# Predecir en el conjunto de prueba
y_pred_logistic = best_logistic_model.predict(X_test_scaled)

# Calcular la exactitud del modelo
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print('Exactitud del modelo de Regresión Logística:', accuracy_logistic.round(4))

<p> 8.2 Modelo para inferir el tipo de transacción que pueden realizar los usuarios de la red.</p>
<h2> Random Forest </h2>

In [ ]:
df_1[['Nombre Transaccion', 'ID. Transaccion']].value_counts()

In [ ]:
df_2[['ID. Transaccion']].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features (X) y target (y)
X = df_2.drop('ID. Transaccion', axis=1)  # Excluir la columna de 'ID. Transaccion'
y = df_2['ID. Transaccion']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Random Forest
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_rf = random_forest_model.predict(X_test)

# Calcular la exactitud del modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print('Exactitud del modelo de Random Forest:', accuracy_rf)

# Mostrar el reporte de clasificación
print('Reporte de clasificación:\n', classification_report(y_test, y_pred_rf))

<p> 8.3 Modelo para predecir los retiros en efectivo y de esta manera provicionar el cash necesario para cubrir las necesidades de los usuarios </p>
<h3> Regresión Lineal - Arboles de decision</h3> 

In [ ]:
# Crear df_4 con las filas de df_2 donde 'ID. Transaccion' está en la lista [3, 4, 14, 19]
transacciones_filtradas = [3, 4, 14, 19]
df_4 = df_2[df_2['ID. Transaccion'].isin(transacciones_filtradas)].copy()

df_4 = df_4[(df_4['Codigo Entidad Orgien'].astype(str).str.len() <= 3) & (df_4['Codigo Entidad Orgien'] != 0)]

In [ ]:
valores = df_4['Valor']

# Crear la gráfica de caja de bigotes
plt.boxplot(valores)
plt.xlabel('Columna Valor')
plt.ylabel('Valor')
plt.title('Gráfico de Caja de Bigotes para la Columna Valor')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Features (X) y target (y)
X = df_4.drop('Valor', axis=1)  # Excluir la columna de 'Valor'
y = df_4['Valor']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Regresión Lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = linear_model.predict(X_test)

# Calcular métricas de regresión (en este caso usamos r2_score y mean_squared_error)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('Coeficiente de determinación (R2):', r2)
print('Error cuadrático medio (MSE):', mse)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Features (X) y target (y)
X = df_4.drop('Valor', axis=1)  # Excluir la columna de 'Valor'
y = df_4['Valor']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Regresión de Árbol de Decisiones
decision_tree_model = DecisionTreeRegressor(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = decision_tree_model.predict(X_test)

# Calcular métricas de regresión (en este caso usamos r2_score y mean_squared_error)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('Coeficiente de determinación (R2):', r2)
print('Error cuadrático medio (MSE):', mse)

<h2> 10. Conclusiones </h2>